# SISTEMA DE RECOMENDACIÓN BASADO EN FILTRO COLABORATIVO


El modelo presentado en este notebook es un modelo de filtrado colaborativo que busca la similitud entre usuarios para realizar recomendaciones de productos. El filtrado colaborativo hace recomendaciones basadas en interacciones entre el usuario y el producto en el pasado. La suposición detrás del algoritmo es que a usuarios similares les gustan productos similares.

Un sistema de recomendación funciona tanto para personalizar un producto como para entender las preferencias de los consumidores, su comportamiento y los productos de interes, este sistema ofrece información sobre los steakhouses que probablemente le gusten a consumidores que nunca han visitado esos lugares, asi como también información general sobre el usuario. 

- Como paso fundamental se importan las librerias que utilizarán

In [1]:
from google.cloud import bigquery
import pandas as pd

- Se crea una instancia de Bigquery para extraer los datos a traves de una consulta SQL y luego convertirlo en Dataframe para trabajar los datows. 
- Se extraen datos de tres tablas diferentes

In [2]:
# Instancia del cliente de BigQuery
client = bigquery.Client()

# Consulta en BigQuery
query = """
SELECT 
  stars, business_id, user_id
FROM 
  `prueba-2-400914.ml.review`
"""

# Se convierte en dataframe
df = client.query(query).to_dataframe()


print(df.head())

   stars             business_id                 user_id
0      1  Ktw3a8hXOKL00UCk_1nSEQ  X7yr3gouERBJMdYwg76t5w
1      1  E0KwLVaQ-uKEjJl3j_VAGQ  njixyfrztdnY7o_1hXgiZQ
2      1  0JPi9cyV9i3_kDPj049_qQ  GS76fpDQjwyoZfVafbjdnw
3      1  W4rgF_1eKzFZiGUOsgbGMA  kT6fw89RSFNNU8U1pyG-cQ
4      1  seKihQKpGGnCeLuELRQPSQ  kbnf0BQnww-wm7Zf_ciwdg


In [4]:
# Consulta en BigQuery
query = """
SELECT 
  name, categories, business_id, latitude, longitude, city, address
FROM 
  `prueba-2-400914.ml.business`
"""

# Lo convierto en Dataframe
df_business = client.query(query).to_dataframe()


print(df_business.head())

                            name  \
0              TD Bank Havertown   
1                 Lott's Service   
2        Beaver Hill Condominium   
3           Jim's Auto Detailing   
4  Creature Comforts Pet Sitting   

                                          categories             business_id  \
0          Banks & Credit Unions, Financial Services  FC6ef4rMMZKNov_-A9M6iQ   
1                   Gas Stations, Automotive, Towing  DcUDIlxFsvqPvFh8_lb4Sw   
2             Real Estate, Apartments, Home Services  UoKkgWmE9-c3wbWwO4VAVQ   
3  Automotive, Auto Detailing, Car Wash, Auto Gla...  do6EpBc7ZUoaqdzuUuGFXA   
4                    Pet Services, Pet Sitting, Pets  8ThWgxiTB83nrIGbZQnNBg   

    latitude  longitude        city                 address  
0  39.985661 -75.314844   Haverford          120 W Eagle Rd  
1  39.973753 -74.995949  Moorestown  908 N Lenola Rd, Ste 3  
2  40.094306 -75.135247  Jenkintown        309 Florence Ave  
3  40.047520 -75.409145       Wayne     729 W Lancaste

In [3]:
# Consulta en BigQuery
query = """
SELECT 
  user_id, review_count, average_stars
FROM 
  `prueba-2-400914.ml.user`
"""

# Lo convierto en Dataframe
df_user = client.query(query).to_dataframe()


print(df_user.head())

                  user_id  review_count  average_stars
0  E9kcWJdJUHuTKfQurPljwA           358           3.73
1  lO1iq-f75hnPNZkTy3Zerg            40           4.04
2  x7YtLnBW2dUnrrpwaofVQQ            37           3.84
3  QF1Kuhs8iwLWANNZxebTow           607           4.11
4  aF3mEXDJuILmeF-9PcxAsQ           250           3.53


- Se unen las tablas para trabajar con todos los datos

In [6]:
df = pd.merge(df_business, df, on='business_id')
print(df.shape)

(88917, 9)


In [7]:
df = pd.merge(df_user, df, on='user_id')
print(df.shape)

(88917, 11)


In [72]:
df.isna().sum()

user_id          0
review_count     0
average_stars    0
name             0
categories       0
business_id      0
latitude         0
longitude        0
city             0
address          0
stars            0
dtype: int64

In [46]:
df.head()

,user_id,review_count,average_stars,name,categories,business_id,stars
0,XLs_PhrJ7Qwn_RfgMM7Djw,90,3.62,Firebirds Wood Fired Grill,"Seafood, American (New), Steakhouses, Nightlife, Restaurants, Bars, Burgers, Wine Bars",luUGLqA3Gxv5raCJOAzblw,4
1,HoiOETUtwO4CL0PhjDCnSw,177,3.85,Chuck's of Hawaii,"Salad, Steakhouses, Restaurants, Nightlife, Seafood",ERzIS-WRLw-80_B4yqcG3Q,4
2,K7thO1n-vZ9PFYiC7nTR2w,1554,3.68,Geno's Steaks,"Sandwiches, Cheesesteaks, Steakhouses, Restaurants",IkY2ticzHEn4QFn8hQLSWg,4
3,UQFE3BT1rsIYrcDvu_XVow,1560,4.17,Geno's Steaks,"Sandwiches, Cheesesteaks, Steakhouses, Restaurants",IkY2ticzHEn4QFn8hQLSWg,5
4,UQFE3BT1rsIYrcDvu_XVow,1560,4.17,Dickie Brennan's Steakhouse,"Restaurants, Cajun/Creole, Steakhouses, Seafood, Local Flavor",C26xmwM2_OgJJi2BbL3KkQ,4


In [47]:
df["business_id"].nunique()

1479

- Se utilizan los datos solo de los usuarios que han realizado más de 5 calificaciones en distintos restaurantes, de lo contrario el modelo no tendría suficiente datos para realizar las comparaciones entre usuarios.

In [8]:
filtered_results = []
grouped_df = df.groupby('user_id')
for user_id, group in grouped_df:
    unique_business_ids = group['business_id'].nunique()
    if unique_business_ids  > 5:
        filtered_results.extend(group.values.tolist())

filtered_df = pd.DataFrame(filtered_results, columns=df.columns)
print(filtered_df.shape)

(6093, 11)


- Se crea la siguiente función para asignar una clave numérica a cada ID del usuario, para manejar mejor los datos

In [9]:
def asignar_clave_numerica(columna):
    claves = {}
    clave_actual = 1
    resultado = []

    for fila in columna:
        if fila not in claves:
            claves[fila] = clave_actual
            clave_actual += 1
        resultado.append(claves[fila])

    return resultado
filtered_df["user_id"] = asignar_clave_numerica(filtered_df["user_id"])

In [10]:
filtered_df.shape

(6093, 11)

Se crea una Matriz donde las columnas son los restaurantes y las filas son los usuarios, el contenido de cada fila será la calificación que el usuario le ha dando al restaurante. En caso de no existir calificación se mostrará NaN.

- En este tipo de modelos es normal que los datos útiles sean escasos, sin embargo es posible predecir la calificación de un usuario a traves de tecnicas como; factorización de matriz o correlación de pearson.

In [33]:
matrix = filtered_df.pivot_table(index='user_id', columns='name', values='stars')
matrix.head()

name,1200 Chophouse,13 Restaurant,14 West Restaurant & Suites,1818 Chophouse,1904 Steakhouse,1933 Lounge,39 North Taproom & Grill,400 Beach Seafood & Tap House,7 Ounce Korean Steakhouse,801 Chophouse,...,Wiz Kid,Woodfire MJ,Woody's Steak House,Yamato Japanese Steakhouse & Sushi Bar,Yamato Japanese Sushi and Hibachi,Yard House,Ybor Chophouse,Yuki Japanese Steak House,Zambelli's Prime Rib Steak & Pizza,Zoes Kitchen
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- El siguiente paso es medir la similitud de los usuarios. En este caso se usa la correlación de pearson

In [34]:
user_similarity = matrix.T.corr()
user_similarity.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,658,659,660,661,662,663,664,665,666,667
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,-1.0,NaN,NaN,-1.0,NaN,-0.944911,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,-1.0,-0.349215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En este paso se usa el ID de usuario 11 como ejemplo para ilustrar cómo encontrar usuarios similares.

Primero debemos excluir el ID de usuario 11 de la lista de usuarios similares y decidir la cantidad de usuarios similares.

In [35]:
# Seleccionar usuario
userid = 11

# Quitar el usuario de la lista
user_similarity.drop(index=userid, inplace=True)

# mostrar similitudes
user_similarity.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,658,659,660,661,662,663,664,665,666,667
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,-1.0,NaN,NaN,-1.0,NaN,-0.944911,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,-1.0,-0.349215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En la matriz de similitud del usuario, los valores varían de -1 a 1, donde -1 significa similitud opuesta y 1 significa similitud igual.

n = 5 significa los 5 usuarios más similares para el ID de usuario 11.

El filtrado colaborativo basado en usuarios hace recomendaciones basadas en usuarios con gustos similares, por lo que debemos establecer un umbral positivo. Aquí configuramos user_similarity_threshold en 0,3, lo que significa que un usuario debe tener un coeficiente de correlación de Pearson de al menos 0,3 para ser considerado un usuario similar.

Después de establecer el número de usuarios similares y el umbral de similitud, clasificamos el valor de similitud del usuario del más alto al más bajo, luego imprimimos la identificación de los usuarios más similares y el valor de correlación de Pearson.

In [38]:
# PRUEBA CON LOS DATOS FILTRADOS POR BUSINESS
# Numero de usuarios similares
n = 5

# Umbral
user_similarity_threshold = 0.3

# Obtener el top n de usuarios similares
similar_users = user_similarity[user_similarity[userid]>user_similarity_threshold][userid].sort_values(ascending=False)[:n]

# Ver Usuarios similares
print(f'Los usuarios similares para el usuario {userid} son ', similar_users)

Los usuarios similares para el usuario 11 son  user_id
229    1.000000
601    1.000000
114    1.000000
370    0.866025
Name: 11, dtype: float64


El siguiente paso es reducir el grupo de elementos para simular una recomendación, se hace lo siguiente:

Eliminar los negocios que haya visitado el usuario seleccionado (ID de usuario 11 en este ejemplo).
Conservar sólo las calificaciones de usuarios similares
Para realizar lo puntos mencionados se mantiene en la fila solo al usuario seleccionado y se elimina los valores faltantes

In [39]:
# PRUEBA CON LOS DATOS FILTRADOS POR BUSINESS
picked_userid = matrix[matrix.index == userid].dropna(axis=1, how='all')
picked_userid

name,Atlantis Steakhouse,Bimini Steakhouse,Cactus Creek Prime Steakhouse,Fin & Filet,Harrah's Steak House,Nevada Steak,Rapscallion Seafood House & Bar,Roxy,The Steak House at Western Village,Wild River Grille
user_id,,,,,,,,,,
11,4.0,3.0,4.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0


In [41]:
# business_id que los usuarios similares han calificado 
similar_user_business = matrix[matrix.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_business

name,Atlantis Steakhouse,Bimini Steakhouse,Cactus Creek Prime Steakhouse,Chandlers,Charlie Palmer Steak Reno,Claim Jumper Steakhouse & Bar,Harrah's Steak House,Hobey's Casino & Restaurant,Ijji 2 Sushi & Japanese Steakhouse,Prime Rib Grill,Rapscallion Seafood House & Bar,Ruby River Steakhouse,Ruth's Chris Steak House,Steak & Pasta House,Sterling's Seafood Steakhouse,The Steak House at Western Village,The Steakhouse Grill,Wild River Grille
user_id,,,,,,,,,,,,,,,,,,
114,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,3.0,4.0,NaN,NaN
229,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,4.0
370,4.0,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
601,NaN,NaN,4.0,5.0,NaN,NaN,5.0,5.0,NaN,NaN,4.0,NaN,5.0,NaN,NaN,5.0,4.0,NaN


In [42]:
# Elimina los negocios que el usuario seleccionado ha visitado
similar_user_business.drop(picked_userid.columns,axis=1, inplace=True, errors='ignore')

# Tdata
similar_user_business

name,Chandlers,Charlie Palmer Steak Reno,Claim Jumper Steakhouse & Bar,Hobey's Casino & Restaurant,Ijji 2 Sushi & Japanese Steakhouse,Prime Rib Grill,Ruby River Steakhouse,Ruth's Chris Steak House,Steak & Pasta House,Sterling's Seafood Steakhouse,The Steakhouse Grill
user_id,,,,,,,,,,,
114,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,3.0,NaN
229,NaN,NaN,4.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,3.0
370,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0
601,5.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN,4.0


En este paso se obtiene el top 3 de recomendaciones que se realizan al usuario seleccionado; 11.

Los elementos recomendados están determinados por el promedio ponderado de la puntuación de similitud del usuario y la calificación del Restauran. Las calificaciones del restauran se ponderan según las puntuaciones de similitud, por lo que los usuarios con mayor similitud obtienen ponderaciones más altas.

Este código recorre los elementos y los usuarios para obtener la puntuación del elemento, clasificar la puntuación de mayor a menor y elegir las 3 mejores restaurantes para recomendar al usuario 11.

In [44]:
# PRUEBA CON LOS DATOS FILTRADOS POR BUSINESS

# Se declara un diccionario para el puntaje
item_score = {}

# recorrer Restaurantes
for i in similar_user_business.columns:
  # calificaciones de los Restaurantes
  business = similar_user_business[i]
  #  variable que almacena los puntajes
  total = 0
  # variable que almacena el numero de puntajes
  count = 0
  # recorrer usuarios similares
  for u in similar_users.index:
    #Si el restauran tiene calificación
    if pd.isna(business[u]) == False:
      # La puntuación es la suma de la puntuación de similitud del usuario multiplicada por la calificación del restaurante
      score = similar_users[u] * business[u]
      # suma puntuacion a la puntuacion total
      total += score
      # Agrega uno al conteo
      count +=1
  # obtiene la puntuacion media del item
  item_score[i] = total / count

# Convertir diccionario en un dataframe
item_score = pd.DataFrame(item_score.items(), columns=['business', 'score'])
    
# Ordenar
ranked_item_score = item_score.sort_values(by='score', ascending=False)

# Seleccionar top 10
m = 3
ranked_item_score["business"].head(m)

0                      Chandlers
3    Hobey's Casino & Restaurant
7       Ruth's Chris Steak House
Name: business, dtype: object

Se crea una funcion para aplicar el mismo procedimiento a cada uno de los usuarios y colocar las recomendaciones en una nueva columna del dataframe.

In [78]:
def recommend_restaurants(userid):

    
    n = 3

    # Umbral
    user_similarity_threshold = 0.3
  
    similar_users = user_similarity[user_similarity[userid]>user_similarity_threshold][userid].sort_values(ascending=False)[:n]
    similar_user_business = matrix[matrix.index.isin(similar_users.index)].dropna(axis=1, how='all')    
    
   # picked_userid = matrix[matrix.index == userid].dropna(axis=1, how='all')
    # similar_user_business.drop(picked_userid.columns,axis=1, inplace=True, errors='ignore')
    
    item_score = {}

    # recorrer Restaurantes
    for i in similar_user_business.columns:
        # calificaciones de los Restaurantes
        business = similar_user_business[i]
        # variable que almacena los puntajes
        total = 0
        # variable que almacena el numero de puntajes
        count = 0
        # recorrer usuarios similares
        for u in similar_users.index:
            # Si el restaurante tiene calificación
            if pd.isna(business[u]) == False:
                # La puntuación es la suma de la puntuación de similitud del usuario multiplicada por la calificación del restaurante
                score = similar_users[u] * business[u]
                # suma puntuacion a la puntuacion total
                total += score
                # Agrega uno al conteo
                count += 1
        # obtiene la puntuacion media del item
        item_score[i] = total / count

    # Ordenar por puntuación en orden descendente
    ranked_item_score = sorted(item_score.items(), key=lambda x: x[1], reverse=True)

    # Seleccionar top 10
    m = 3
    top_3_businesses = [business for business, score in ranked_item_score[:m]]

    return top_3_businesses

In [79]:
filtered_df["recomendaciones"] = filtered_df['user_id'].apply(recommend_restaurants)

In [33]:
pd.set_option('max_colwidth', 600)
filtered_df["recomendaciones"]

0       [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
1       [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
2       [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
3       [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
4       [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
                                          ...                                    
6088                        [Chop Steakhouse & Bar, Hardware Grill, Harvest Room]
6089                        [Chop Steakhouse & Bar, Hardware Grill, Harvest Room]
6090                        [Chop Steakhouse & Bar, Hardware Grill, Harvest Room]
6091                        [Chop Steakhouse & Bar, Hardware Grill, Harvest Room]
6092                        [Chop Steakhouse & Bar, Hardware Grill, Harvest Room]
Name: recomendaciones, Length: 6093, dtype: object

- Se exportan los datos a un bucket de Google Storage para luego usarlos en la aplicación Streamlit

In [102]:
#EXPORTAR DATOS DE PRUEBA


from google.cloud import storage

bucket_nombre = 'archivo_ml'
parquet_file_nombre = 'data_sistema_final.parquet'

filtered_df.to_parquet(parquet_file_nombre, index=False)

client = storage.Client()

bucket = client.get_bucket(bucket_nombre)
blob = bucket.blob(parquet_file_nombre)
blob.upload_from_filename(parquet_file_nombre)

- Funciones para Streamlit

In [103]:
def obtener_recomendaciones(user_id):
    # Filtrar el DataFrame por user_id
    df_filtrado = df_prueba[df_prueba['user_id'] == user_id]

    if df_filtrado.empty:
        return 'No se encontraron recomendaciones para el user_id especificado.'

    # Obtener las recomendaciones (tomando la primera fila ya que son iguales para cada user_id)
    recomendaciones = df_filtrado.iloc[0]['recomendaciones']

    return recomendaciones

In [36]:
obtener_recomendaciones(1)

0    [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
1    [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
2    [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
3    [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
4    [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
5    [Tee-Off Restaurant and Lounge, Holdren's Steaks & Seafood, Jill's Place]
Name: recomendaciones, dtype: object

In [90]:
def obtener_informacion(user_id):
    # Filtrar el DataFrame por user_id
    df_filtrado = filtered_df[filtered_df['user_id'] == user_id]

    if df_filtrado.empty:
        return 'No se encontraron recomendaciones para el user_id especificado.'


    # Obtener el promedio de las estrellas y el número total de reseñas
    promedio_estrellas = df_filtrado.iloc[0]['average_stars']
    total_resenas = df_filtrado.iloc[0]['review_count']

    # Contar el número de reseñas en steakhouses
    numero_resenas_steakhouses = filtered_df[filtered_df['user_id'] == user_id].shape[0]

    # Crear un diccionario con las recomendaciones, el promedio de estrellas, el número total de reseñas y el número de reseñas en steakhouses
    resultado = {
        'Promedio de Calificación': promedio_estrellas,
        'Rseñas Totales': total_resenas,
        'Reseñas Totales en Steakhouses': numero_resenas_steakhouses
    }

    return resultado

In [91]:
obtener_informacion(6)

{'Promedio de Calificación': 3.99,
 'Rseñas Totales': 174,
 'Reseñas Totales en Steakhouses': 6}

In [94]:
def obtener_direccion_recomendadas(user_id):
    address_1 = []
    df_usuario = filtered_df[filtered_df['user_id'] == user_id]
    recomendaciones = df_usuario['recomendaciones'].iloc[0]
    for restaurante in recomendaciones:
        df_restaurante = filtered_df[filtered_df['name'] == restaurante]
        if not df_restaurante.empty:
            city = df_restaurante["city"].iloc[0]
            address = df_restaurante['address'].iloc[0]
            address_1.append((city, address))
    return address_1

In [95]:
obtener_direccion_recomendadas(1)

[('Santa Barbara', '3627 State St'),
 ('Santa Barbara', '512 State St'),
 ('Santa Barbara', '632 Santa Barbara St')]

In [96]:
filtered_df["address_recommend"] = filtered_df['user_id'].apply(obtener_direccion_recomendadas)

In [112]:

#FUNCION PARA LA APP DE STREAMLIT

def obtener_recomendaciones(user_id):
    # Filtrar el DataFrame por user_id
    df_filtrado = filtered_df[filtered_df['user_id'] == user_id]

    if df_filtrado.empty:
        return 'No se encontraron recomendaciones para el user_id especificado.'

    # Obtener las recomendaciones y direcciones
    recomendaciones = df_filtrado.iloc[0]['recomendaciones']
    direcciones = df_filtrado.iloc[0]['address_recommend']

    # Crear una lista de recomendaciones con sus direcciones
    recomendaciones_con_direcciones = []
    for recomendacion, direccion in zip(recomendaciones, direcciones):
        recomendacion_con_direccion = f"{recomendacion}, ubicado en; {direccion[0]}, {direccion[1]}"
        recomendaciones_con_direcciones.append(recomendacion_con_direccion)

    return recomendaciones_con_direcciones

In [115]:
obtener_recomendaciones(9)

["Bynum's Steakhouse, ubicado en; Indianapolis, 3850 S Meridian St",
 "Charleston's, ubicado en; Indianapolis, 6815 E 82nd St",
 "Fleming's Prime Steakhouse & Wine Bar - Indianapolis, ubicado en; Indianapolis, 8487 Union Chapel Rd, Ste 120"]

In [120]:
def obtener_nombres(user_id):
    nombres = []
    df_usuario = filtered_df[filtered_df['user_id'] == user_id]
    recomendaciones = df_usuario['recomendaciones'].iloc[0]
    for restaurante in recomendaciones:
        df_restaurante = filtered_df[filtered_df['name'] == restaurante]
        if not df_restaurante.empty:
            nombre = df_restaurante['name'].iloc[0]
            nombres.append(nombre)
    return nombres

En la aplicación de Streamlit se pueden arrojar diferentes ubicaciones para un mismo restaurante, esto pasa porque hay restaurantes pertenecientes a frenquicias.

In [132]:
mascara = df["name"] == "Ruth's Chris Steak House"

In [133]:
df[mascara]

,user_id,review_count,average_stars,name,categories,business_id,latitude,longitude,city,address,stars,recomendaciones,address_recommend,coordenadas_recommend
25,3,291,4.16,Ruth's Chris Steak House,"Steakhouses, Restaurants",25,43.616142,-116.203274,Boise,"800 W Main St, Ste 110",5,"[Aqua Prime, Harold Seltzer's Steak House, Ruth's Chris Steak House]","[[Indian Rocks Beach, 213 Gulf Blvd], [Saint Petersburg, 3500 Tyrone Blvd N], [Boise, 800 W Main St, Ste 110]]","[[-82.8499127276, 27.8797124687], [-82.7449549, 27.8031306], [-116.203274, 43.6161421]]"
35,4,642,3.30,Ruth's Chris Steak House,"Steakhouses, Restaurants",35,39.925402,-86.110951,Indianapolis,9445 Threel Rd,4,"[Bern's Steak House, Chandlers, Charley's Steak House]","[[Tampa, 1208 S Howard Ave], [Boise, 981 W Grove St], [Tampa, 4444 West Cypress Street]]","[[-82.4827004, 27.9318624], [-116.2064158, 43.6155352], [-82.519544, 27.951538]]"
44,5,1019,3.68,Ruth's Chris Steak House,"Steakhouses, Restaurants",44,39.766275,-86.159539,Indianapolis,"45 S. Illinois Street, Circle Centre Mall",3,"[Eddie Merlot's - Indianapolis, Fogo de Chao Brazilian Steakhouse, Morton's The Steakhouse]","[[Indianapolis, 3645 E 96th St], [Indianapolis, 117 E Washington St], [St. Louis, 999 N 2nd St]]","[[-86.1042061, 39.9264824], [-86.1555469969, 39.7667119938], [-90.18409, 38.63285]]"
45,5,1019,3.68,Ruth's Chris Steak House,"Restaurants, Steakhouses",45,39.911980,-86.116269,Indianapolis,2727 E 86th St,3,"[Eddie Merlot's - Indianapolis, Fogo de Chao Brazilian Steakhouse, Morton's The Steakhouse]","[[Indianapolis, 3645 E 96th St], [Indianapolis, 117 E Washington St], [St. Louis, 999 N 2nd St]]","[[-86.1042061, 39.9264824], [-86.1555469969, 39.7667119938], [-90.18409, 38.63285]]"
84,8,1454,4.21,Ruth's Chris Steak House,"American (Traditional), Restaurants, Bars, Steakhouses, Wine Bars, Seafood, Nightlife",80,27.957381,-82.524796,Tampa,1700 N Westshore Blvd,4,"[Cazador Grill, Eddie V's Prime Seafood, Ocean Prime]","[[Temple Terrace, 10918 N 56th St], [Tampa, 4400 W Boy Scout Blvd], [Indianapolis, 8555 N River Rd 100, Ste 100]]","[[-82.3942414, 28.0481089], [-82.5183134644, 27.9602024049], [-86.105556246, 39.9115059354]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5949,651,275,3.99,Ruth's Chris Steak House,"American (Traditional), Restaurants, Bars, Steakhouses, Wine Bars, Seafood, Nightlife",80,27.957381,-82.524796,Tampa,1700 N Westshore Blvd,5,"[Del Frisco's Grille, PY Steakhouse, Outback Steakhouse]","[[Tampa, 4138 W Boy Scout Blvd], [Tucson, 5655 W Valencia Rd], [Indianapolis, 3454 W 86th St]]","[[-82.5136891007, 27.9644574661], [-111.0868383, 32.1312887], [-86.2202697, 39.9117677]]"
5950,651,275,3.99,Ruth's Chris Steak House,"American (Traditional), Restaurants, Bars, Steakhouses, Wine Bars, Seafood, Nightlife",80,27.957381,-82.524796,Tampa,1700 N Westshore Blvd,5,"[Del Frisco's Grille, PY Steakhouse, Outback Steakhouse]","[[Tampa, 4138 W Boy Scout Blvd], [Tucson, 5655 W Valencia Rd], [Indianapolis, 3454 W 86th St]]","[[-82.5136891007, 27.9644574661], [-111.0868383, 32.1312887], [-86.2202697, 39.9117677]]"
5973,654,370,4.11,Ruth's Chris Steak House,"Steakhouses, Restaurants",44,39.766275,-86.159539,Indianapolis,"45 S. Illinois Street, Circle Centre Mall",5,"[Hyde Park Prime Steakhouse, Fleming's Prime Steakhouse & Wine Bar - Indianapolis, Ruth's Chris Steak House]","[[Indianapolis, 51 N Illinois St], [Indianapolis, 8487 Union Chapel Rd, Ste 120], [Boise, 800 W Main St, Ste 110]]","[[-86.159615, 39.768401], [-86.1061134961, 39.9098017418], [-116.203274, 43.6161421]]"
5985,656,112,4.25,Ruth's Chris Steak House,"Steakhouses, Restaurants",44,39.766275,-86.159539,Indianapolis,"45 S. Illinois Street, Circle Centre Mall",5,"[Barclay Prime, Butcher Bar, Butcher and Singer]","[[Philadelphia, 237 S 18th St], [Philadelphia, 2034 Chestnut St], [Philadelphia, 1500 Walnut St]]","[[-75.1708743, 39.9487096], [-75.1748960093, 39.9520301], [-75.1661758, 39.9493338]]"
